In [ ]:
!pip install segmentation-models-pytorch

In [2]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn
import torch

# 'mit_b0' Mix Vision Transformer Backbone from SegFormer pretrained on Imagenet
preprocess_input = get_preprocessing_fn('mit_b0', pretrained='imagenet')

# MixVisionTransformer encoder does not support in_channels setting other than 3
#  supported by FPN only for encoder depth = 5
model = smp.FPN('mit_b0', in_channels=3, classes = 10, encoder_depth = 5) 

# ... Train model on your dataset

mask = model(torch.randn([1, 3, 64, 64]))
print(mask.shape) # torch.Size([1, 1, 64, 64])

Downloading: "https://github.com/qubvel/segmentation_models.pytorch/releases/download/v0.0.2/mit_b0.pth" to /root/.cache/torch/hub/checkpoints/mit_b0.pth


  0%|          | 0.00/13.7M [00:00<?, ?B/s]

torch.Size([1, 10, 64, 64])


Train

In [46]:
from torch.nn import CrossEntropyLoss, BCEWithLogitsLoss

dice_loss = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
iou_loss = smp.losses.JaccardLoss(smp.losses.BINARY_MODE, from_logits=True)
bce_loss = BCEWithLogitsLoss()

gt = torch.randint_like(mask,0,2)
print(mask.shape,gt.shape)
print("BCE Loss",bce_loss(mask,gt))
print("DICE Loss",dice_loss(mask,gt))
print("IoU Loss",iou_loss(mask,gt))

torch.Size([1, 10, 64, 64]) torch.Size([1, 10, 64, 64])
BCE Loss tensor(1.5019, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
DICE Loss tensor(0.5199, grad_fn=<MeanBackward0>)
IoU Loss tensor(0.6841, grad_fn=<MeanBackward0>)


In [47]:
ce_loss = CrossEntropyLoss()

gt2, _ = gt.max(dim = 1) 
gt2 = gt2.long() 
print("Mask :",mask.shape,"GT for CE loss",gt2.shape) # 4d vs 3d !
print("CE",ce_loss(mask,gt2))

Mask : torch.Size([1, 10, 64, 64]) GT for CE loss torch.Size([1, 64, 64])
CE tensor(3.4609, grad_fn=<NllLoss2DBackward0>)
